## [Computational Social Science] Project 5: Natural Language Processing

### Team 4: Elleni Hailu, Lawrence Liu, and Ángel Mendiola Ross
### Due: February 26, 2020

In this project, we use natural language processing techniques to explore a dataset containing tweets from members of the 116th United States Congress that met from January 3, 2019 to January 2, 2021. The dataset has also been cleaned to contain information about each legislator. Concretely, we do the following:

* Preprocess the text of legislators' tweets
* Conduct Exploratory Data Analysis of the text
* Use sentiment analysis to explore differences between legislators' tweets
* Featurize text with manual feature engineering, frequency-based, and vector-based techniques
* Predict legislators' political parties and whether they are a Senator or Representative

We explore two questions that relate to two central findings in political science and examine how they relate to the text of legislators' tweets. First, political scientists have argued that U.S. politics is currently highly polarized relative to other periods in American history, but also that the polarization is asymmetric. Historically, there were several conservative Democrats (i.e. "blue dog Democrats") and liberal Republicans (i.e. "Rockefeller Republicans"), as measured by popular measurement tools like [DW-NOMINATE](https://en.wikipedia.org/wiki/NOMINATE_(scaling_method)#:~:text=DW\%2DNOMINATE\%20scores\%20have\%20been,in\%20the\%20liberal\%2Dconservative\%20scale.). However, in the last few years, there are few if any examples of any Democrat in Congress being further to the right than any Republican and vice versa. At the same time, scholars have argued that this polarization is mostly a function of the Republican party moving further right than the Democratic party has moved left. **Does this sort of asymmetric polarization show up in how politicians communicate to their constituents through tweets?**

Second, the U.S. Congress is a bicameral legislature, and there has long been debate about partisanship in the Senate versus the House. The House of Representatives is apportioned by population and all members serve two year terms. In the Senate, each state receives two Senators and each Senator serves a term of six years. For a variety of reasons (smaller chamber size, more insulation from the voters, rules and norms like the filibuster, etc.), the Senate has been argued to be the "cooling saucer" of Congress in that it is more bipartisan and moderate than the House. **Does the theory that the Senate is more moderate have support in Senators' tweets?**

**Note**: See the project handout for more details on caveats and the data dictionary.

In [97]:
#install spacy language detect capability, uncomment below if not yet installed 
#!pip install spacy-langdetect

In [98]:
# pandas and numpy
import pandas as pd
import numpy as numpy

# punctuation, stop words and English language model
from string import punctuation

from spacy.lang.en.stop_words import STOP_WORDS
from spacy_langdetect import LanguageDetector #so we can add a feature to the dataframe indicating whether English text or not

import en_core_web_sm
nlp = en_core_web_sm.load()

#add language detection to spaCy pipeline
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

# textblob
from textblob import TextBlob

# countvectorizer, tfidfvectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# gensim
import gensim
from gensim import models

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#replicabilty
import random

#regex to help remove graphical emojis
import re 

In [99]:
random.seed(49764) #set seed for replicability w/ sample

congress_tweets = pd.read_csv("data/116th Congressional Tweets and Demographics.csv")
# fill in this line of code with a sufficient number of tweets, depending on your computational resources
congress_tweets = congress_tweets.sample(5000) 
#LL: note I picked 5000 arbitrarily for now; we'll want to come back to this and adjust up as we get code to work
congress_tweets.head()

,tweet_id,screen_name,datetime,text,name_wikipedia,position,joined_congress_date,birthday,gender,state,district_number,party,trump_2016_state_share,clinton_2016_state_share,obama_2012_state_share,romney_2012_state_share
305871,1.296130e+18,RepCasten,2020-08-19T12:54:53-04:00,RT @NeoCrp @RepCasten When prioritizing future...,Sean Casten,Rep,3-Jan-19,11/23/1971,M,IL,6,Democrat,"2,146,015","3,090,729","3,019,512","2,135,216"
731734,1.278730e+18,WhipClyburn,2020-07-02T12:25:12-04:00,The Civil Rights Act of 1964 was key to starti...,Jim Clyburn,Rep,3-Jan-93,7/21/1940,M,SC,6,Democrat,"1,155,389","855,373","865,941","1,071,645"
531575,1.294360e+18,RepDonaldPayne,2020-08-14T15:35:29-04:00,@flynn_chick I am. The Post Office is a natio...,Donald Payne Jr.,Rep,15-Nov-12,12/17/1958,M,NJ,10,Democrat,"1,601,933","2,148,278","2,125,101","1,477,568"
60337,1.093980e+18,RepSwalwell,2019-02-08T15:53:56-05:00,RT @kylegriffin1 After ranking member Doug Col...,Eric Swalwell,Rep,3-Jan-13,11/16/1980,M,CA,15,Democrat,"4,483,814","8,753,792","7,854,285","4,839,958"
36066,1.275170e+18,RepDavid,2020-06-22T17:09:45-04:00,Sources: Below are resources on different ways...,David Schweikert,Rep,3-Jan-13,3/3/1962,M,AZ,6,Republican,"1,252,401","1,161,167","1,025,232","1,233,654"


In [100]:
congress_tweets.shape

(5000, 16)

## Preprocessing Text Data

The first step in working with text data is to preprocess it. Make sure you do the following:

* Remove punctuation and stop words. The `rem_punc_stop()` function we used in lab is provided to you but you should feel free to edit it as necessary for other steps
* Remove tokens that occur frequently in tweets, but may not be helpful for downstream classification. For instance, many tweets contain a flag for retweeting, or share a URL 

As you search online, you might run into solutions that rely on regular expressions. You are free to use these, but you should also be able to preprocess using the techniques we covered in lab. Specifically, we encourage you to use spaCy's token attributes and string methods to do some of this text preprocessing.

In [101]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [102]:
#function to remove graphical emojis
#copied from: https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python/50602709#50602709

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [103]:
def rem_punc_stop(text):
    stop_words = STOP_WORDS
    punc = set(punctuation) #should get rid of hashtags and @s
    
    punc_free = "".join([ch for ch in text if ch not in punc]) #turns text into a string object
    
    punc_free = punc_free.lower() #lower-case the words
    
    punc_free = punc_free.replace('rt', '') #delete rt
    
    punc_free = deEmojify(punc_free) #remove graphical emojis
    
    punc_free = " ".join(punc_free.split()) #removes all whitespace characters (space, line breaks, etc.)

    #punc_free = punc_free.replace('\n', '') #remove line breaks
    
    doc = nlp(punc_free)
    
    spacy_words = [token.text for token in doc]
    
    spacy_words = [word for word in spacy_words if not word.startswith('http')]
    
    no_punc = [word for word in spacy_words if word not in stop_words]
    
    return no_punc

In [104]:
congress_tweets['tokens'] = congress_tweets['text'].map(lambda x: rem_punc_stop(x))
congress_tweets['tokens']

305871    [neocrp, repcasten, prioritizing, future, regu...
731734    [civil, rights, act, 1964, key, staing, cascad...
531575    [flynnchick, post, office, national, instituti...
60337     [kylegriffin1, ranking, member, doug, collins,...
36066     [sources, resources, different, ways, buy, sel...
                                ...                        
43998              [worrying, information, care, neighbors]
872214    [president, innocent, let, allow, testimony, k...
694803    [defeat, coronavirus, contract, disease, mild,...
145052    [horrific, accusations, forced, hysterectomies...
472985    [mikepence, proud, repo, challenging, time, na...
Name: tokens, Length: 5000, dtype: object

## Adding Feature Indicating the Tweet Language

Since some of the tweets are in non-English languages, we added a column to the dataframe that indicates the language of the tweet as identified by spaCy. 

In [105]:
#example of how it works and what is output
test_text = ['.', 'this is a test string']
for text in test_text:
    doc = nlp(text)
    print(doc._.language)

{'language': 'UNKNOWN', 'score': 0.0}
{'language': 'en', 'score': 0.9999984220475613}


In [106]:
#code adapted from: https://www.kaggle.com/shawamar/language-detection-using-spacy
#LL note 2/14/21: note that w/ 5000 tweets, this took me ~3 minutes to run

#create container for tweet languages and loop through text column of dataframe
tweets = congress_tweets['text']
languages_spacy = []

for text in tweets:
    doc = nlp(text)
    languages_spacy.append(list(doc._.language.values())[0])

#add column to df indicating language of tweet
congress_tweets['languages_spacy'] = languages_spacy

In [107]:
congress_tweets.head()

,tweet_id,screen_name,datetime,text,name_wikipedia,position,joined_congress_date,birthday,gender,state,district_number,party,trump_2016_state_share,clinton_2016_state_share,obama_2012_state_share,romney_2012_state_share,tokens,languages_spacy
305871,1.296130e+18,RepCasten,2020-08-19T12:54:53-04:00,RT @NeoCrp @RepCasten When prioritizing future...,Sean Casten,Rep,3-Jan-19,11/23/1971,M,IL,6,Democrat,"2,146,015","3,090,729","3,019,512","2,135,216","[neocrp, repcasten, prioritizing, future, regu...",en
731734,1.278730e+18,WhipClyburn,2020-07-02T12:25:12-04:00,The Civil Rights Act of 1964 was key to starti...,Jim Clyburn,Rep,3-Jan-93,7/21/1940,M,SC,6,Democrat,"1,155,389","855,373","865,941","1,071,645","[civil, rights, act, 1964, key, staing, cascad...",en
531575,1.294360e+18,RepDonaldPayne,2020-08-14T15:35:29-04:00,@flynn_chick I am. The Post Office is a natio...,Donald Payne Jr.,Rep,15-Nov-12,12/17/1958,M,NJ,10,Democrat,"1,601,933","2,148,278","2,125,101","1,477,568","[flynnchick, post, office, national, instituti...",en
60337,1.093980e+18,RepSwalwell,2019-02-08T15:53:56-05:00,RT @kylegriffin1 After ranking member Doug Col...,Eric Swalwell,Rep,3-Jan-13,11/16/1980,M,CA,15,Democrat,"4,483,814","8,753,792","7,854,285","4,839,958","[kylegriffin1, ranking, member, doug, collins,...",en
36066,1.275170e+18,RepDavid,2020-06-22T17:09:45-04:00,Sources: Below are resources on different ways...,David Schweikert,Rep,3-Jan-13,3/3/1962,M,AZ,6,Republican,"1,252,401","1,161,167","1,025,232","1,233,654","[sources, resources, different, ways, buy, sel...",en


## Exploratory Data Analysis

Use two of the techniques we covered in lab (or other techniques outside of lab!) to explore the text of the tweets. You should construct these visualizations with an eye toward the eventual classification tasks: (1) predicting the legislator's political party based on the text of their tweet, and (2) predicting whether the legislator is a Senator or Representative. As a reminder, in lab we covered word frequencies, word clouds, word/character counts, scattertext, and topic modeling as possible exploration tools. 

### EDA 1

### EDA 2

## Sentiment Analysis

Next, let's analyze the sentiments contained within the tweets. You may use TextBlob or another library for these tasks. Do the following:

* Choose two legislators, one who you think will be more liberal and one who you think will be more conservative, and analyze their sentiment and/or subjectivity scores per tweet. For instance, you might do two scatterplots that plot each legislator's sentiment against their subjectivity, or two density plots for their sentiments. Do the scores match what you thought?
* Plot two more visualizations like the ones you chose in the first part, but do them to compare (1) Democrats v. Republicans and (2) Senators v. Representatives 

`TextBlob` has already been imported in the top cell.

## Featurization

Before going to classification, explore different featurization techniques. Create three dataframes or arrays to represent your text features, specifically:

* Features engineered from your previous analysis. For example, word counts, sentiment scores, topic model etc.
* A term frequency-inverse document frequency matrix. 
* An embedding-based featurization (like a document averaged word2vec)

In the next section, you will experiment with each of these featurization techniques to see which one produces the best classifications.

### Engineered Text Features

In [ ]:
# Engineered Features
...

### Bag-of-words or Tf-idf

In [ ]:
# Frequency Based featurization
...

### Word Embedding

In [ ]:
# Load Word2Vec model from Google; OPTIONAL depending on your computational resources (the file is ~1 GB)
# Also note that this file path assumes that the word vectors are underneath 'data'; you may wish to point to the CSS course repo and change the path
# or move the vector file to the project repo 

#model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz', binary = True) 

In [ ]:
# Function to average word embeddings for a document; use examples from lab to apply this function. You can use also other techniques such as PCA and doc2vec instead.
def document_vector(word2vec_model, doc):
    doc = [word for word in doc if word in model.vocab]
    return np.mean(model[doc], axis=0)

In [ ]:
# embedding based featurization
...

## Classification

Either use cross-validation or partition your data with training/validation/test sets for this section. Do the following:

* Choose a supervised learning algorithm such as logistic regression, random forest etc. 
* Train six models. For each of the three dataframes you created in the featurization part, train one model to predict whether the author of the tweet is a Democrat or Republican, and a second model to predict whether the author is a Senator or Representative.
* Report the accuracy and other relevant metrics for each of these six models.
* Choose the featurization technique associated with your best model. Combine those text features with non-text features. Train two more models: (1) A supervised learning algorithm that uses just the non-text features and (2) a supervised learning algorithm that combines text and non-text features. Report accuracy and other relevant metrics. 

If time permits, you are encouraged to use hyperparameter tuning or AutoML techniques like TPOT, but are not explicitly required to do so.

### Train Six Models with Just Text

In [ ]:
# six models ([engineered features, frequency-based, embedding] * [democrat/republican, senator/representative])
...

### Two Combined Models

In [ ]:
# two models ([best text features + non-text features] * [democrat/republican, senator/representative])
...

## Discussion Questions

1. Why do standard preprocessing techniques need to be further customized to a particular corpus?

**Your Answer Here**

2. Did you find evidence for the idea that Democrats and Republicans have different sentiments in their tweets? What about Senators and Representatives?

**Your Answer Here**

3. Why is validating your exploratory and unsupervised learning approaches with a supervised learning algorithm valuable?

**Your Answer Here**

4. Did text only, non-text only, or text and non-text features together perform the best? What is the intuition behind combining text and non-text features in a supervised learning algorithm?

**Your Answer Here**